In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob
import librosa
import pandas as pd

import torchaudio
from speechbrain.pretrained.interfaces import foreign_class

torchvision is not available - cannot save figures


In [2]:
classifier = foreign_class(source="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier", savedir="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", run_opts={"device":"cuda"})
accents = ["us", "england", "australia", "indian", "canada", "bermuda", "scotland", "african", "ireland", "newzealand", "wales", "malaysia", "philippines", "singapore", "hongkong", "southatlandtic",]

tempfile = "temp.wav"
def run_classification(path):
    audio, _ = librosa.load(path, sr=fs)
    torchaudio.save(tempfile, torch.tensor(audio).unsqueeze(0), fs)
    signal, org_sr = torchaudio.load(tempfile)
    signal = torchaudio.functional.resample(signal, orig_freq=org_sr, new_freq=fs)
    outputs =  classifier.encode_batch(signal)
    embeddings =  outputs[0].detach().cpu().numpy()
    outputs = classifier.mods.output_mlp(outputs)
    rawaccent = outputs[0].detach().cpu().numpy()
    probaccent = classifier.hparams.softmax(outputs).detach().cpu().numpy()[0]
    return embeddings, rawaccent, probaccent

def cosine_similarity(e1, e2): # from wespeaker, delete the normalizing part
    e1 = torch.tensor(e1)
    e2 = torch.tensor(e2)
    cosine_score = torch.dot(e1, e2) / (torch.norm(e1) * torch.norm(e2))
    cosine_score = cosine_score.item()
    return cosine_score

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [3]:
fs = 16000
accent_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"
native_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/English/wav/"
base_model_dir = "/mntcephfs/lab_data/shoinoue/Models/trained_models/AC_01/ckpts_16000/"
model_paths = [
#     "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_smaller/100000",
#     "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
#     "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_smaller/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",
    
#     "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_smaller/100000",
#     "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
#     "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_smaller/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
#     "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    
#     "VTN_fine-tuning_nocondition_gt2syn_hifiganmelhifiganmel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_hifiganmelhifiganmel_hubert_norepeating/100000",
]
base_dataset_dir = "/mntcephfs/lab_data/shoinoue/Dataset/"
dataset_paths = [
    # "PD-AST/SLT/English",
    # "PD-AST/SLT/Hindi",
    # "PD-AST/SLT/Korean",
    # "PD-AST/ASI/Hindi",
    # "PD-AST/TNI/Hindi",
    # "PD-AST/HKK/Korean",
    # "PD-AST/YDCK/Korean",
    # "CMU-ARCTIC/SLT",
    # "L2-ARCTIC/ASI",
    # "L2-ARCTIC/TNI",
    # "L2-ARCTIC/HKK",
    # "L2-ARCTIC/YDCK",
]

In [10]:
save_dir = f"evaluation/accent/"
testsamples = list(np.load("data_split_ARCTIC.npy", allow_pickle=True))[2]
for ij in range(2):
    base_dir = [base_model_dir, base_dataset_dir][ij]
    dirnames = [model_paths, dataset_paths][ij]
    for dirname in dirnames:
        save_path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
        # if os.path.exists(save_path):
            # continue
        print(dirname)
        files = glob.glob(base_dir + f"{dirname}/wav/*.wav")
        files.sort()
        raws, probs, embs = [], [], []
        for path in tqdm(files):
            basename = os.path.basename(path)[:-4]
            if not(basename in testsamples):
                continue
            accent_path = f"./evaluation/accent_embeddings/Hindi/{basename}.npy"
            native_path = f"./evaluation/accent_embeddings/English/{basename}.npy"
            accent = np.load(accent_path, allow_pickle=True).item()
            native = np.load(native_path, allow_pickle=True).item()
            
            embeddings, rawaccent, probaccent = run_classification(path)
            a = cosine_similarity(embeddings, accent["embeddings"])
            b = cosine_similarity(embeddings, native["embeddings"])
            diff = a-b
            raws += [rawaccent[3]]
            probs += [probaccent[3]]
            embs += [diff]
            # if len(embs)>5:
            #     break

        # np.save(save_path, [raws, probs, embs])

VTN_fine-tuning_nocondition_mix2synVCTK3hr_hifiganmelhifiganmel_hubert_norepeating/100000


100%|██████████| 100/100 [00:05<00:00, 17.59it/s]
